## Sweep bot

i'm fucking crashing out
why is nothing fucking working

In [1]:
%load_ext autoreload
%autoreload 2
# ^ make it so we can change inner files


In [2]:

import numpy as np
from pydrake.all import (
    AddDefaultVisualization,
    ConstantVectorSource,
    ContactModel,
    Demultiplexer,
    DiagramBuilder,
    InverseDynamicsController,
    MeshcatVisualizer,
    MultibodyPlant,
    Rgba,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    SceneGraph,
    Simulator,
    Sphere,
    StartMeshcat,
    TrajectorySource,
    VectorLogSink,
    VisualizationConfig,
)

from manipulation.meshcat_utils import (
    AddMeshcatTriad,
    PublishPositionTrajectory,
)
from manipulation.station import (
    AddPointClouds,
    MakeHardwareStation,
    Parser,
)
from scripts.load_scenario import load_scenario
from scripts.grasp_broom import plan_path, grasp_path, build_temp_plant
from scripts.meta_controller import MetaController

In [3]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link above to open Meshcat in your browser!


In [4]:
from scripts.final_hail_mary import plan_sweep
from scripts.trajectory_helpers import ManipulateBroom
use_hydroelastic = True # this takes forever

def make_diagram():

    scenario = load_scenario(use_position=True)

    # since makehardwarestation finalizes the plant before giving it back to us
    def callback(parser: Parser):
        plant = parser.plant()
        plant.set_contact_model(ContactModel.kHydroelasticWithFallback)
        scene_graph: SceneGraph = parser.scene_graph()
        config = scene_graph.get_config()
        if use_hydroelastic:
            # one object rigid, all else compliant? table
            # HE modulus parameters
            # allow broom to turn more easily

            # add "handle" to broom to make it easier to grip?
            config.default_proximity_properties.compliance_type = "compliant"
        scene_graph.set_config(config)
        # AddDefaultVisualization(parser.builder(), meshcat)

    station = MakeHardwareStation(scenario, meshcat=meshcat, parser_prefinalize_callback=callback)
    context = station.CreateDefaultContext()
    plant: MultibodyPlant = station.plant()
    controller = MetaController(plant, station)

    builder = DiagramBuilder()
    builder.AddSystem(station)

    to_point_cloud = AddPointClouds(
        scenario=scenario, station=station, builder=builder, meshcat=meshcat
    )
    builder.ExportOutput(
        to_point_cloud["camera0"].point_cloud_output_port(), "point_cloud0"
    )

    plant_context = plant.CreateDefaultContext()
    q0 = plant.GetPositions(plant_context, plant.GetModelInstanceByName('iiwa'))
    broom_frame = plant.GetFrameByName("handle_link")
    X_WStart = broom_frame.CalcPoseInWorld(plant_context)
    X_WGoal = RigidTransform(RotationMatrix(), np.array([0, 0.5, 0]))
    AddMeshcatTriad(meshcat, 'goal', X_PT=X_WGoal)

    v_src = builder.AddSystem(ConstantVectorSource([0]*7))

    wsg_src = builder.AddSystem(ConstantVectorSource([0.5]))

    # NOTE: Intentionally do NOT connect the trajectory source to the iiwa input port.
    # This leaves the iiwa unhooked so it won't be driven by this notebook.
    # You can manually control the broom (its frames/pose) via the plant/context as needed.

    builder.Connect(
        v_src.get_output_port(),
        station.GetInputPort('iiwa.position')
    )

    # builder.ExportOutput(traj_src.get_output_port(), 'pos')

    builder.Connect(
        wsg_src.get_output_port(),
        station.GetInputPort("wsg.position")
    )


    diagram = builder.Build()

    return diagram, plant, station, controller


In [5]:
diagram, plant, station, controller  = make_diagram()

In [10]:

simulator = Simulator(diagram)
context = simulator.get_mutable_context()
controller.add_diagram(diagram, context, meshcat)
sweep_maker = ManipulateBroom()
traj = sweep_maker.trajectory(controller)
traj_end = traj.end_time()
for i in range(11):
    time = i * traj_end / 10
    pose = traj.GetPose(time)
    meshcat.SetTransform(str(i) + "sphere", pose)
    sph = meshcat.SetObject(str(i) + "sphere", Sphere(0.02), Rgba(0, 1, 0, 1))

diagram.ForcedPublish(context)


[[-0.4553426  -0.45335782 -0.45137304 ... -0.28465208 -0.2826673
  -0.28068253]
 [ 0.64664215  0.64664215  0.64664215 ...  0.32709363  0.32709363
   0.32709363]
 [ 0.0499999   0.0499999   0.0499999  ...  0.0499999   0.0499999
   0.0499999 ]]


In [7]:
"""
plant_context = plant.GetMyContextFromRoot(context)
# controller.add_diagram(diagram, context, meshcat)
iiwa = plant.GetModelInstanceByName('iiwa')
broom_inst = plant.GetModelInstanceByName('broom')
broom = plant.GetBodyByName('handle_link', broom_inst)
# iiwa_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("base", iiwa))
# broom_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("handle_link"))
# AddMeshcatTriad(meshcat, 'iiwa', X_PT=iiwa_pose)
# AddMeshcatTriad(meshcat, 'broom', X_PT=broom_pose)

simulator.set_target_realtime_rate(1.0)
meshcat.StartRecording()
simulator.Initialize()
end_time = 3.4
time = 0

while time < end_time:
    time += 0.05
    q = traj.GetPose(time)
    plant.SetFreeBodyPose(plant_context, broom, q)
    diagram.ForcedPublish(context)
    simulator.AdvanceTo(time)
    # time.sleep(0.05)

simulator.AdvanceTo(2)

meshcat.StopRecording()
meshcat.PublishRecording()
# controller.sub_meshcat.StopRecording()
# controller.sub_meshcat.PublishRecording()
"""

'\nplant_context = plant.GetMyContextFromRoot(context)\n# controller.add_diagram(diagram, context, meshcat)\niiwa = plant.GetModelInstanceByName(\'iiwa\')\nbroom_inst = plant.GetModelInstanceByName(\'broom\')\nbroom = plant.GetBodyByName(\'handle_link\', broom_inst)\n# iiwa_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("base", iiwa))\n# broom_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("handle_link"))\n# AddMeshcatTriad(meshcat, \'iiwa\', X_PT=iiwa_pose)\n# AddMeshcatTriad(meshcat, \'broom\', X_PT=broom_pose)\n\nsimulator.set_target_realtime_rate(1.0)\nmeshcat.StartRecording()\nsimulator.Initialize()\nend_time = 3.4\ntime = 0\n\nwhile time < end_time:\n    time += 0.05\n    q = traj.GetPose(time)\n    plant.SetFreeBodyPose(plant_context, broom, q)\n    diagram.ForcedPublish(context)\n    simulator.AdvanceTo(time)\n    # time.sleep(0.05)\n\nsimulator.AdvanceTo(2)\n\nmeshcat.StopRecording()\nmeshcat.PublishRecording()\n# controller.sub_meshca